In [64]:
import yaml
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.font_manager import FontProperties
import time
from scipy.stats import entropy as scipy_entropy

In [65]:
def load_raw_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

In [66]:
def get_parameters():
    return {
        "POPULATION_SIZE": 200,
        "ELITE_PROPORTION": 0.1,
        "MUTANT_PROPORTION": 0.2,
        "RHO_E": 0.5,
        "MAX_GENERATION_NUMBER": 50,
        "RESTART_THRESHOLD": 5,
        "RESTART_PROPORTION": 0.5,
        "IMPROVEMENT_WINDOW": 7,
        "IMPROVEMENT_EPSILON": 0.00005,
        "MAX_FITNESS": 1.0,
        "USE_LOCAL_SEARCH": True,
        "CROSSOVER_MODE": "mix",
        "INIT_MODES": ["combinada", "carga", "aleatorio", "sala_escassa"]
    }


In [67]:

def check_feasibility(raw_data, verbose=True):
    days = raw_data["days"]
    hours = raw_data["hours"]
    rooms = raw_data["rooms"]
    cols = raw_data["course_mapping"]
    classes = raw_data["classes"]

    LEN_CLASSES = len(classes)
    LEN_DAYS = len(days)
    LEN_HOURS = len(hours)

    total_units_needed = sum([units for _, _, _, units in cols]) * LEN_CLASSES
    total_slots_available = LEN_DAYS * LEN_HOURS * sum(rooms["number"])

    if verbose:
        print("\n📊 Análise Geral de Viabilidade:")
        print(f"- Unidades letivas totais necessárias: {total_units_needed}")
        print(f"- Slots totais disponíveis (todas as salas): {total_slots_available}")
        print(f"- Diferença: {total_slots_available - total_units_needed} slots livres\n")

    room_type_to_count = {rtype: qty for rtype, qty in zip(rooms["type"], rooms["number"])}
    required_units_by_room_type = {rtype: 0 for rtype in rooms["type"]}
    available_slots_by_room_type = {}

    for subject, room_type, _, units in cols:
        required_units_by_room_type[room_type] += units * LEN_CLASSES

    if verbose:
        print("🏫 Análise por Tipo de Sala:")
    for room_type, required_units in required_units_by_room_type.items():
        available_slots = room_type_to_count[room_type] * LEN_DAYS * LEN_HOURS
        available_slots_by_room_type[room_type] = available_slots

        if verbose:
            print(f"- {room_type}:")
            print(f"   • Requerido: {required_units} slots")
            print(f"   • Disponível: {available_slots} slots")
            print(f"   • Diferença: {available_slots - required_units}\n")

        if required_units > available_slots:
            raise ValueError(
                f"Alocação impossível para salas do tipo '{room_type}':\n"
                f"Requerido = {required_units} | Disponível = {available_slots}"
            )

    required_units_by_professor = {}
    max_slots_per_professor = LEN_DAYS * LEN_HOURS

    for _, _, professor, units in cols:
        required_units_by_professor.setdefault(professor, 0)
        required_units_by_professor[professor] += units * LEN_CLASSES

    if verbose:
        print("👨‍🏫 Análise por Professor:")
    for professor, required_units in required_units_by_professor.items():
        if verbose:
            print(f"- {professor}:")
            print(f"   • Requerido: {required_units} slots")
            print(f"   • Disponível: {max_slots_per_professor}")
            print(f"   • Diferença: {max_slots_per_professor - required_units}\n")

        if required_units > max_slots_per_professor:
            raise ValueError(
                f"O professor '{professor}' tem carga horária excessiva: "
                f"{required_units} unidades letivas atribuídas, mas só há {max_slots_per_professor} slots disponíveis."
            )

    if verbose:
        print("✅ Alocação viável: recursos suficientes no total, por tipo de sala e por professor.\n")


In [68]:
import numpy as np
from scipy.stats import entropy as scipy_entropy

def get_entropy(individual, class_name, raw_data, threshold=0.0):
    classes = raw_data["classes"]
    cols = raw_data["course_mapping"]
    LEN_COLS = len(cols)
    LEN_DAYS = len(raw_data["days"])
    LEN_HOURS = len(raw_data["hours"])
    class_index = classes.index(class_name)
    u_per_day_mat = np.zeros((LEN_COLS, LEN_DAYS), dtype=np.uint8)

    for day in range(LEN_DAYS):
        for hour in range(LEN_HOURS):
            for row in range(LEN_COLS):
                idx = hour + day * LEN_HOURS + class_index * LEN_DAYS * LEN_HOURS
                if individual["genes"][idx, row] > 0:
                    u_per_day_mat[row, day] += 1

    total_entropy = 0.0
    for row in range(u_per_day_mat.shape[0]):
        row_sum = np.sum(u_per_day_mat[row, :])
        if row_sum > 0:
            entropy = scipy_entropy(u_per_day_mat[row, :], base=2)
            if entropy > threshold:
                total_entropy += entropy
    return total_entropy / LEN_COLS

def get_fragmentation(individual, class_name, raw_data):
    classes = raw_data["classes"]
    cols = raw_data["course_mapping"]
    LEN_DAYS = len(raw_data["days"])
    LEN_HOURS = len(raw_data["hours"])
    LEN_COLS = len(cols)
    class_idx = classes.index(class_name)
    total_fragmentation = 0

    for col_idx in range(LEN_COLS):
        dias_usados = set()
        base = class_idx * LEN_DAYS * LEN_HOURS
        for dia in range(LEN_DAYS):
            for hora in range(LEN_HOURS):
                idx = base + dia * LEN_HOURS + hora
                if individual["genes"][idx, col_idx] > 0:
                    dias_usados.add(dia)
        if len(dias_usados) > 0:
            total_fragmentation += len(dias_usados)
    return total_fragmentation / LEN_COLS

def evaluate(individual, raw_data):
    classes = raw_data["classes"]
    cols = raw_data["course_mapping"]
    LEN_CLASSES = len(classes)
    LEN_DAYS = len(raw_data["days"])
    LEN_HOURS = len(raw_data["hours"])

    scheduled_units = 0
    expected_units = sum([units for _, _, _, units in cols]) * LEN_CLASSES
    unallocated_units = 0

    for col_idx, (_, _, _, units) in enumerate(cols):
        for class_idx in range(LEN_CLASSES):
            base = class_idx * LEN_DAYS * LEN_HOURS
            allocated = sum(individual["genes"][base + i, col_idx] > 0 for i in range(LEN_DAYS * LEN_HOURS))
            if allocated >= units:
                scheduled_units += units
            else:
                scheduled_units += allocated
                unallocated_units += (units - allocated)

    percentual_alocado = scheduled_units / expected_units
    entropia_media = sum(get_entropy(individual, clss, raw_data) for clss in classes) / LEN_CLASSES

    def sigmoid(x):
        return 1 / (1 + np.exp(-10 * (x - 0.5)))

    sig_entropia = sigmoid(percentual_alocado)
    sig_nao_alocadas = sigmoid(percentual_alocado)

    peso_entropia = 0.01 + 0.04 * sig_entropia
    peso_nao_alocadas = 0.25 * (1 - sig_nao_alocadas) + 0.1
    frag_media = sum(get_fragmentation(individual, clss, raw_data) for clss in classes) / LEN_CLASSES
    peso_fragmentacao = 0.05

    fitness = percentual_alocado         - entropia_media * peso_entropia         - (unallocated_units / expected_units) * peso_nao_alocadas         - frag_media * peso_fragmentacao

    return (fitness, percentual_alocado * 100)


def init_individual_variante(modo, raw_data):
    cols = raw_data["course_mapping"]
    rooms = raw_data["rooms"]

    if modo == "aleatorio":
        keys = np.random.rand(len(cols))

    elif modo == "sala_escassa":
        dificuldade = []
        for _, room_type, _, units in cols:
            n_salas = rooms["number"][rooms["type"].index(room_type)]
            dificuldade.append(units / n_salas)
        base_order = sorted(range(len(cols)), key=lambda i: -dificuldade[i])
        keys = np.zeros(len(cols))
        for rank, col_idx in enumerate(base_order):
            keys[col_idx] = (len(cols) - rank) / len(cols)
        noise = np.random.normal(0, 0.1, len(cols))
        keys = np.clip(keys + noise, 0.0, 1.0)

    elif modo == "combinada":
        professor_counts = {}
        for _, _, prof, _ in cols:
            professor_counts[prof] = professor_counts.get(prof, 0) + 1

        prioridades = []
        for _, _, prof, units in cols:
            prioridade = 0.7 * professor_counts[prof] + 0.3 * units
            prioridades.append(prioridade)

        base_order = sorted(range(len(cols)), key=lambda i: -prioridades[i])
        keys = np.zeros(len(cols))
        for rank, col_idx in enumerate(base_order):
            keys[col_idx] = (len(cols) - rank) / len(cols)
        noise = np.random.normal(0, 0.06, len(cols))
        keys = np.clip(keys + noise, 0.0, 1.0)

    else:  # padrão por carga
        base_order = sorted(range(len(cols)), key=lambda i: -cols[i][3])
        keys = np.zeros(len(cols))
        for rank, col_idx in enumerate(base_order):
            keys[col_idx] = (len(cols) - rank) / len(cols)
        noise = np.random.normal(0, 0.05, len(cols))
        keys = np.clip(keys + noise, 0.0, 1.0)

    return keys


def mutate_keys(keys, gen=0, max_gen=100):
    progress = gen / max_gen
    taxa_inversao = 0.1 * (1 - progress) + 0.02 * progress
    taxa_ruido = 0.06 * (1 - progress) + 0.01 * progress
    desvio = 0.03 * (1 - progress) + 0.01 * progress

    mutated = keys.copy()
    n = len(keys)

    for _ in range(int(taxa_inversao * n)):
        i, j = random.sample(range(n), 2)
        if abs(mutated[i] - mutated[j]) < 0.15:
            mutated[i], mutated[j] = mutated[j], mutated[i]

    for i in range(n):
        if random.random() < taxa_ruido:
            mutated[i] += np.random.normal(0, desvio)

    mutated = np.clip(mutated, 0.0, 1.0)
    return mutated

def biased_crossover(parent1, parent2, rho_e=0.5):
    return np.where(np.random.rand(len(parent1)) < rho_e, parent1, parent2)

def interpolated_crossover(parent1, parent2, alpha=0.5):
    noise = np.random.normal(0, 0.01, len(parent1))
    return np.clip(alpha * parent1 + (1 - alpha) * parent2 + noise, 0.0, 1.0)

def crossover_variavel(p1, p2, rho_e=0.5, alpha=0.5, modo="mix"):
    if modo == "biased":
        return biased_crossover(p1, p2, rho_e)
    elif modo == "interpolated":
        return interpolated_crossover(p1, p2, alpha)
    else:  # "mix" ou qualquer outro valor
        if random.random() < 0.7:
            return biased_crossover(p1, p2, rho_e)
        else:
            return interpolated_crossover(p1, p2, alpha)

    
def torneio_binario(populacao, raw_data):
    a, b = random.sample(populacao, 2)
    return a if evaluate(a, raw_data)[0] > evaluate(b, raw_data)[0] else b

In [69]:
def decode_individual(keys, raw_data, use_local_search=True):
    cols = raw_data["course_mapping"]
    rooms = raw_data["rooms"]
    days = raw_data["days"]
    hours = raw_data["hours"]
    classes = raw_data["classes"]

    LEN_ROWS = len(classes) * len(days) * len(hours)
    LEN_COLS = len(cols)
    LEN_ROOMS = sum(rooms["number"])

    chromosome = list(np.argsort(keys))
    individual = {
        "keys": keys,
        "chromosome": chromosome,
        "available_room": np.zeros((len(days) * len(hours), LEN_ROOMS), dtype=np.int8),
        "genes": np.zeros((LEN_ROWS, LEN_COLS), dtype=np.int32)
    }

    ind = fill_genes_brkga(individual, raw_data)
    if use_local_search:
        return hill_climbing_melhorar_blocos(ind, raw_data)
    else:
        return ind


def fill_genes_brkga(ind, raw_data):
    cols = raw_data["course_mapping"]
    rooms = raw_data["rooms"]
    days = raw_data["days"]
    hours = raw_data["hours"]
    classes = raw_data["classes"]

    LEN_CLASSES = len(classes)
    LEN_DAYS = len(days)
    LEN_HOURS = len(hours)

    professor_schedule = {}

    for working_column in ind["chromosome"]:
        subject, room_type, lecturer, units = cols[working_column]

        for clss in range(LEN_CLASSES):
            start = clss * LEN_DAYS * LEN_HOURS
            allocated_units = 0
            dias_utilizados = set()

            for dia in range(LEN_DAYS):
                if len(dias_utilizados) >= 2 and dia not in dias_utilizados:
                    continue

                for hora in range(LEN_HOURS - units + 1):
                    bloco_viavel = all(
                        not professor_schedule.get((dia, hora + offset, lecturer), False)
                        for offset in range(units)
                    )
                    if not bloco_viavel:
                        continue

                    bloco_completo = True
                    for offset in range(units):
                        row = start + dia * LEN_HOURS + hora + offset
                        tipo_idx = rooms["type"].index(room_type)
                        s = sum(rooms["number"][:tipo_idx])
                        e = s + rooms["number"][tipo_idx] - 1

                        if not any(
                            ind["genes"][row, working_column] == 0 and
                            ind["available_room"][row % (LEN_DAYS * LEN_HOURS), room] == 0
                            for room in range(s, e + 1)
                        ):
                            bloco_completo = False
                            break

                    if not bloco_completo:
                        continue

                    # Realiza alocação
                    for offset in range(units):
                        hora_atual = hora + offset
                        row = start + dia * LEN_HOURS + hora_atual
                        tipo_idx = rooms["type"].index(room_type)
                        s = sum(rooms["number"][:tipo_idx])
                        e = s + rooms["number"][tipo_idx] - 1

                        for room in range(s, e + 1):
                            if ind["genes"][row, working_column] == 0 and ind["available_room"][row % (LEN_DAYS * LEN_HOURS), room] == 0:
                                ind["genes"][row, working_column] = room + 1
                                ind["available_room"][row % (LEN_DAYS * LEN_HOURS), room] = 1
                                professor_schedule[(dia, hora_atual, lecturer)] = True

                                base = row % (LEN_DAYS * LEN_HOURS)
                                for clss2 in range(LEN_CLASSES):
                                    if clss2 != clss:
                                        ind["genes"][base + clss2 * LEN_DAYS * LEN_HOURS, working_column] = -1
                                for col in range(len(cols)):
                                    if col != working_column:
                                        ind["genes"][row, col] = -1
                                break

                    allocated_units += units
                    dias_utilizados.add(dia)
                    break

                if allocated_units >= units:
                    break

            # Fallback
            if allocated_units < units:
                for dia in range(LEN_DAYS):
                    for hora in range(LEN_HOURS):
                        if allocated_units >= units:
                            break
                        row = start + dia * LEN_HOURS + hora
                        if professor_schedule.get((dia, hora, lecturer), False):
                            continue

                        tipo_idx = rooms["type"].index(room_type)
                        s = sum(rooms["number"][:tipo_idx])
                        e = s + rooms["number"][tipo_idx] - 1

                        for room in range(s, e + 1):
                            if ind["genes"][row, working_column] == 0 and ind["available_room"][row % (LEN_DAYS * LEN_HOURS), room] == 0:
                                ind["genes"][row, working_column] = room + 1
                                ind["available_room"][row % (LEN_DAYS * LEN_HOURS), room] = 1
                                professor_schedule[(dia, hora, lecturer)] = True

                                base = row % (LEN_DAYS * LEN_HOURS)
                                for clss2 in range(LEN_CLASSES):
                                    if clss2 != clss:
                                        ind["genes"][base + clss2 * LEN_DAYS * LEN_HOURS, working_column] = -1
                                for col in range(len(cols)):
                                    if col != working_column:
                                        ind["genes"][row, col] = -1

                                allocated_units += 1
                                break
    return ind


In [70]:
def hill_climbing_melhorar_blocos(ind, raw_data, max_tentativas=30):
    classes = raw_data["classes"]
    cols = raw_data["course_mapping"]
    rooms = raw_data["rooms"]
    LEN_CLASSES = len(classes)
    LEN_DAYS = len(raw_data["days"])
    LEN_HOURS = len(raw_data["hours"])
    LEN_COLS = len(cols)

    professor_schedule = {
        (dia, hora, cols[col_idx][2])
        for row in range(LEN_CLASSES * LEN_DAYS * LEN_HOURS)
        for col_idx in range(LEN_COLS)
        if ind["genes"][row, col_idx] > 0
        for dia in [row // (LEN_HOURS * LEN_CLASSES) % LEN_DAYS]
        for hora in [row // LEN_CLASSES % LEN_HOURS]
    }

    class_schedule = np.zeros((LEN_CLASSES, LEN_DAYS, LEN_HOURS), dtype=bool)
    for row in range(LEN_CLASSES * LEN_DAYS * LEN_HOURS):
        dia = (row // LEN_HOURS) % LEN_DAYS
        hora = row % LEN_HOURS
        clss = row // (LEN_DAYS * LEN_HOURS)
        if any(ind["genes"][row, col] > 0 for col in range(LEN_COLS)):
            class_schedule[clss, dia, hora] = True

    for tentativa in range(max_tentativas):
        for col_idx, (subject, room_type, lecturer, units) in enumerate(cols):
            for class_idx in range(LEN_CLASSES):
                base = class_idx * LEN_DAYS * LEN_HOURS
                already_allocated = sum(ind["genes"][base + i, col_idx] > 0 for i in range(LEN_DAYS * LEN_HOURS))
                if already_allocated >= units:
                    continue

                unidades_restantes = units - already_allocated
                for dia in range(LEN_DAYS):
                    for hora in range(LEN_HOURS - unidades_restantes + 1):
                        conflito = any(
                            (dia, hora + offset, lecturer) in professor_schedule or
                            class_schedule[class_idx, dia, hora + offset]
                            for offset in range(unidades_restantes)
                        )
                        if conflito:
                            continue

                        linhas = [base + dia * LEN_HOURS + hora + offset for offset in range(unidades_restantes)]
                        tipo_sala_idx = rooms["type"].index(room_type)
                        sala_inicio = sum(rooms["number"][:tipo_sala_idx])
                        sala_fim = sala_inicio + rooms["number"][tipo_sala_idx]

                        for room in range(sala_inicio, sala_fim):
                            if all(ind["available_room"][linha % (LEN_DAYS * LEN_HOURS), room] == 0 for linha in linhas):
                                for linha in linhas:
                                    ind["genes"][linha, col_idx] = room + 1
                                    ind["available_room"][linha % (LEN_DAYS * LEN_HOURS), room] = 1

                                    hora_local = (linha % (LEN_DAYS * LEN_HOURS)) % LEN_HOURS
                                    dia_local = (linha % (LEN_DAYS * LEN_HOURS)) // LEN_HOURS
                                    professor_schedule.add((dia_local, hora_local, lecturer))
                                    class_schedule[class_idx, dia_local, hora_local] = True
                                break
    return ind


In [71]:
def run_brkga(parametros, raw_data):
    modos = parametros.get("INIT_MODES", ["combinada", "carga"])

    POP_SIZE = parametros["POPULATION_SIZE"]
    ELITE_SIZE = int(parametros["ELITE_PROPORTION"] * POP_SIZE)
    MUTANT_SIZE = int(parametros["MUTANT_PROPORTION"] * POP_SIZE)
    MAX_GEN = parametros["MAX_GENERATION_NUMBER"]
    MAX_FITNESS = parametros["MAX_FITNESS"]
    RHO_E = parametros["RHO_E"]
    RESTART_THRESHOLD = parametros["RESTART_THRESHOLD"]
    RESTART_PROPORTION = parametros["RESTART_PROPORTION"]
    IMPROVEMENT_WINDOW = parametros["IMPROVEMENT_WINDOW"]
    IMPROVEMENT_EPSILON = parametros["IMPROVEMENT_EPSILON"]
    USE_LOCAL_SEARCH = parametros.get("USE_LOCAL_SEARCH", True)
    CROSSOVER_MODE = parametros.get("CROSSOVER_MODE", "mix")

    population = [
        decode_individual(init_individual_variante(random.choice(modos), raw_data), raw_data, use_local_search=USE_LOCAL_SEARCH)
        for _ in range(POP_SIZE)
    ]

    best_solution = None
    best_fitness = -np.inf
    stagnation_counter = 0
    fitness_history = []
    
    

    for generation in range(MAX_GEN):
        population.sort(key=lambda ind: evaluate(ind, raw_data)[0], reverse=True)
        elites = population[:ELITE_SIZE]
        mutants = [decode_individual(init_individual_variante(random.choice(modos), raw_data), raw_data, use_local_search=USE_LOCAL_SEARCH)
                   for _ in range(MUTANT_SIZE)]

        offspring = []
        while len(offspring) < POP_SIZE - ELITE_SIZE - MUTANT_SIZE:
            elite = torneio_binario(elites, raw_data)
            non_elite = torneio_binario(population[ELITE_SIZE:], raw_data)

            child_keys = crossover_variavel(elite["keys"], non_elite["keys"], RHO_E, modo=CROSSOVER_MODE)
            child_keys = mutate_keys(child_keys, generation, MAX_GEN)
            child = decode_individual(child_keys, raw_data, use_local_search=USE_LOCAL_SEARCH)

            if evaluate(child, raw_data)[0] > evaluate(population[-1], raw_data)[0] or random.random() < 0.1:
                offspring.append(child)

        population = elites + offspring + mutants
        current_best = population[0]
        current_fitness, allocation_pct = evaluate(current_best, raw_data)
        fitness_history.append(current_fitness)

        print(f"Geração {generation + 1}: Fitness = {current_fitness:.4f} | Alocação = {allocation_pct:.2f}%")

        if current_fitness > best_fitness:
            best_fitness = current_fitness
            best_solution = current_best
            stagnation_counter = 0
        else:
            stagnation_counter += 1

        if stagnation_counter >= RESTART_THRESHOLD:
            n_replace = int(RESTART_PROPORTION * POP_SIZE)
            novos = [
                decode_individual(init_individual_variante(random.choice(modos), raw_data), raw_data, use_local_search=USE_LOCAL_SEARCH)
                for _ in range(n_replace)
            ]
            population[-n_replace:] = novos
            stagnation_counter = 0
            print("🔁 Reinicialização parcial da população.")

        if current_fitness >= MAX_FITNESS:
            break

        if len(fitness_history) >= IMPROVEMENT_WINDOW:
            delta = max(fitness_history[-IMPROVEMENT_WINDOW:]) - min(fitness_history[-IMPROVEMENT_WINDOW:])
            if delta < IMPROVEMENT_EPSILON:
                print(f"📉 Convergência detectada (Δfitness < {IMPROVEMENT_EPSILON})")
                break

    return best_solution

In [72]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd

def get_time_table(individual, class_name, raw_data):
    days = raw_data["days"]
    hours = raw_data["hours"]
    classes = raw_data["classes"]
    cols = raw_data["course_mapping"]
    rooms = raw_data["rooms"]

    LEN_DAYS = len(days)
    LEN_HOURS = len(hours)
    LEN_COLS = len(cols)

    class_idx = classes.index(class_name)
    time_table = pd.DataFrame(index=hours, columns=days)

    for day_idx, day_name in enumerate(days):
        for hour_idx, hour_label in enumerate(hours):
            gene_index = hour_idx + day_idx * LEN_HOURS + class_idx * LEN_DAYS * LEN_HOURS
            for col_idx in range(LEN_COLS):
                if individual["genes"][gene_index, col_idx] > 0:
                    subject, room_type, lecturer, _ = cols[col_idx]
                    room_type_idx = rooms["type"].index(room_type)
                    room_offset = sum(rooms["number"][:room_type_idx])
                    room_number = individual["genes"][gene_index, col_idx] - 1 - room_offset
                    value = f"{subject}, {lecturer}, {room_type}{room_number + 1}"

                    if pd.isna(time_table.loc[hour_label, day_name]):
                        time_table.loc[hour_label, day_name] = value
                    break
    return time_table


def plot_time_table(time_table, title):
    tab20 = ListedColormap(plt.cm.tab20.colors)
    subjects = sorted(set(cell.split(",")[0].strip() for row in time_table.values for cell in row if isinstance(cell, str)))
    subject_color_mapping = {subject: tab20(i % 20) for i, subject in enumerate(subjects)}

    fig, ax = plt.subplots(figsize=(14, len(time_table)))
    ax.set_title(title, fontsize=20)
    ax.axis('off')

    cell_text = [
        [val.replace(',', '\n') if isinstance(val, str) else '' for val in row]
        for row in time_table.values
    ]
    cell_colors = [
        [subject_color_mapping.get(val.split(',')[0].strip(), 'white') if isinstance(val, str) else 'white'
         for val in row]
        for row in time_table.values
    ]

    table = ax.table(
        cellText=cell_text,
        cellLoc='center',
        colLabels=time_table.columns,
        rowLabels=time_table.index,
        cellColours=cell_colors,
        bbox=[0, 0, 1, 1]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.5)

    for (row, col), cell in table.get_celld().items():
        if row == 0 or col == -1:
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
        else:
            text = cell.get_text().get_text()
            if text:
                parts = text.split("\n")
                first_part = parts[0]
                remaining = ' - '.join(parts[1:])
                cell.get_text().set_text(first_part + '\n\n' + remaining)

    plt.tight_layout()
    plt.show()


In [73]:
from itertools import product

def gerar_parametros_experimentos():
    base = get_parameters()

    tamanhos_pop = [100, 150]               # Tamanhos de população
    geracoes = [5, 10, 20]                  # Diferentes profundidades evolutivas
    cruzamentos = ["mix", "biased"]         # Estratégias de cruzamento
    busca_local = [True, False]             # Com e sem busca local

    configuracoes = []
    for pop, gen, cross, local in product(tamanhos_pop, geracoes, cruzamentos, busca_local):
        params = base.copy()
        params["POPULATION_SIZE"] = pop
        params["MAX_GENERATION_NUMBER"] = gen
        params["CROSSOVER_MODE"] = cross
        params["USE_LOCAL_SEARCH"] = local
        configuracoes.append(params)

    return configuracoes


def benchmark_varias_configuracoes(
    lista_arquivos,
    lista_parametros,
    n_execucoes=3
):
    resultados_gerais = []
    melhores_individuos = {}

    for idx, parametros in enumerate(lista_parametros):
        print(f"\n⚙️ Configuração {idx + 1}/{len(lista_parametros)}")
        for arquivo in lista_arquivos:
            print(f"\n📂 Executando para: {arquivo}")
            raw_data = load_raw_data(arquivo)
            check_feasibility(raw_data, verbose=False)

            resultados = []
            melhor_individuo = None
            melhor_fitness = -np.inf

            for i in range(n_execucoes):
                print(f"   🏁 Execução {i + 1}/{n_execucoes}")
                inicio = time.time()
                individuo = run_brkga(parametros, raw_data)
                tempo = time.time() - inicio
                fitness, alocacao = evaluate(individuo, raw_data)

                resultados.append({
                    "instancia": arquivo,
                    "execucao": i + 1,
                    "fitness": fitness,
                    "alocacao": alocacao,
                    "tempo": tempo,
                    "pop_size": parametros["POPULATION_SIZE"],
                    "geracoes": parametros["MAX_GENERATION_NUMBER"],
                    "cross": parametros["CROSSOVER_MODE"],
                    "local_search": parametros["USE_LOCAL_SEARCH"]
                })

                if fitness > melhor_fitness:
                    melhor_fitness = fitness
                    melhor_individuo = individuo

            df_resultado = pd.DataFrame(resultados)
            resultados_gerais.append(df_resultado)

            chave = f"{arquivo} | CFG#{idx + 1}"
            melhores_individuos[chave] = melhor_individuo

            print(f"   ✅ Melhor fitness: {melhor_fitness:.4f}")

    df_final = pd.concat(resultados_gerais, ignore_index=True)

    # 📊 Visualização por parâmetro
    for metrica in ["fitness", "alocacao", "tempo"]:
        plt.figure(figsize=(12, 5))
        df_final.boxplot(column=metrica, by=["instancia", "geracoes"], grid=False)
        plt.title(f"Comparação: {metrica} por Instância x Gerações")
        plt.suptitle("")
        plt.xlabel("Instância x Geração")
        plt.ylabel(metrica.capitalize())
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    return df_final, melhores_individuos


In [ ]:
# Lista de arquivos YAML a comparar
arquivos_yaml = [
    "raw_data_facil.yml",
    "raw_data_medio.yml",
    "raw_data_dificil.yml"
]

param_configs = gerar_parametros_experimentos()
df_resumo_cfgs, melhores_cfgs = benchmark_varias_configuracoes(
    arquivos_yaml,
    lista_parametros=param_configs,
    n_execucoes=3
)




⚙️ Configuração 1/24

📂 Executando para: raw_data_facil.yml
   🏁 Execução 1/3
Geração 1: Fitness = 0.9500 | Alocação = 100.00%
Geração 2: Fitness = 0.9500 | Alocação = 100.00%
Geração 3: Fitness = 0.9500 | Alocação = 100.00%
Geração 4: Fitness = 0.9500 | Alocação = 100.00%
Geração 5: Fitness = 0.9500 | Alocação = 100.00%
   🏁 Execução 2/3
Geração 1: Fitness = 0.9500 | Alocação = 100.00%
Geração 2: Fitness = 0.9500 | Alocação = 100.00%
Geração 3: Fitness = 0.9500 | Alocação = 100.00%
Geração 4: Fitness = 0.9500 | Alocação = 100.00%
Geração 5: Fitness = 0.9500 | Alocação = 100.00%
   🏁 Execução 3/3
Geração 1: Fitness = 0.9500 | Alocação = 100.00%
Geração 2: Fitness = 0.9500 | Alocação = 100.00%
Geração 3: Fitness = 0.9500 | Alocação = 100.00%
Geração 4: Fitness = 0.9500 | Alocação = 100.00%
Geração 5: Fitness = 0.9500 | Alocação = 100.00%
   ✅ Melhor fitness: 0.9500

📂 Executando para: raw_data_medio.yml
   🏁 Execução 1/3
Geração 1: Fitness = 0.9421 | Alocação = 100.00%
Geração 2: Fitne

In [ ]:
for nome_arquivo, individuo in melhores_cfgs.items():
    raw_data = load_raw_data(nome_arquivo.split(" | ")[0])
    for turma in raw_data["classes"]:
        tt = get_time_table(individuo, turma, raw_data)
        plot_time_table(tt, title=f"{turma} ({nome_arquivo})")
